In [ ]:
import os
from glob import glob
from PIL import Image
import numpy as np
import json
import random
from copy import deepcopy
from tqdm import tqdm
import cv2

In [ ]:
# YOLO 형식 txt 파일 생성 함수
def crop_label(crop_h,crop_w,label,input_size):
    # Crop image
    #args: crop_h, crop_w, image, label, input_size
    temp_labels = []
    for k in range(len(label['annotations'])):
        x = label['annotations'][k]['bbox'][0]
        y = label['annotations'][k]['bbox'][1]
        w = label['annotations'][k]['bbox'][2]
        h = label['annotations'][k]['bbox'][3]
        if x >= crop_w+4 and y >= crop_h+4 and x <= crop_w + input_size-4 and y <= crop_h + input_size-4:
            label['annotations'][k]['bbox'][0] = (x - crop_w) 
            label['annotations'][k]['bbox'][1] = (y - crop_h) 
            temp_labels.append(label['annotations'][k])
    return temp_labels

def create_yolo_txt(crop_labels, filename, class_name, input_size=512):
    """
    YOLO 형식의 라벨 txt 파일 생성
    Args:
        crop_labels: 크롭된 라벨 리스트
        filename: 이미지 파일명 (확장자 제외)
        class_name: 'train' or 'val'
    """
    txt_path = f'../../data/yolo_IGNITE/labels/{class_name}/{filename}.txt'
    
    yolo_lines = []
    for label in crop_labels:
        # COCO bbox: [x, y, width, height] (절대좌표)
        # YOLO bbox: [x_center, y_center, width, height] (정규화된 0~1 사이 값)
        
        x, y, w, h = label['bbox']
        
        # 중심점 계산 (이미 정규화된 상태)
        x_center = x + w/2
        y_center = y + h/2
        
        # 클래스 ID (COCO는 1부터 시작, YOLO는 0부터 시작)
        class_id = label['category_id'] - 1
        
        # YOLO 형식: class_id x_center y_center width height
        yolo_line = f"{class_id} {max(0,min(1,x_center/input_size)):.6f} {max(0,min(1,y_center/input_size)):.6f} {max(0,min(1,w/input_size)):.6f} {max(0,min(1,h/input_size)):.6f}"
        yolo_lines.append(yolo_line)
    
    # txt 파일 저장
    with open(txt_path, 'w') as f:
        f.write('\n'.join(yolo_lines))

def custom_yolo_creation(json_data, file_path, input_size=512, class_name='train'):
    """
    YOLO 데이터셋 생성 (이미지 + txt 라벨)
    """
    w = json_data['image']['width']
    h = json_data['image']['height']
    r = input_size / min(h, w)
    image = Image.open(os.path.join(file_path, json_data['image']['file_name']))
    
    base_filename = json_data['image']['file_name'].split('.')[0]
    
    if r < 1:
        # 이미지가 input_size보다 큰 경우 → 크롭
        h_count = h // input_size
        w_count = w // input_size
        
        for hi in range(h_count):
            for wi in range(w_count):
                h1 = hi * input_size
                w1 = wi * input_size
                
                # 경계 조정
                if h1 + input_size > h:
                    h1 = h - input_size
                if w1 + input_size > w:
                    w1 = w - input_size
                
                # 라벨 크롭
                crop_labels = crop_label(h1, w1, json_data, input_size)
                
                if len(crop_labels) > 0:
                    # 이미지 크롭 및 저장
                    crop_image = image.crop((w1, h1, w1 + input_size, h1 + input_size))
                    img_filename = f'{base_filename}_{hi}_{wi}'
                    crop_image.save(f'../../data/yolo_IGNITE/images/{class_name}/{img_filename}.png')
                    
                    # YOLO txt 라벨 생성
                    create_yolo_txt(crop_labels, img_filename, class_name)
                    
    
    else:
        # 이미지가 input_size보다 작거나 같은 경우 → 패딩
        h1, w1 = 0, 0
        crop_labels = crop_label(h1, w1, json_data, input_size)
        
        if len(crop_labels) > 0:
            # 패딩된 이미지 생성
            pad_image = np.ones((input_size, input_size, 3), dtype=np.uint8) * 255
            pad_image[:min(h, input_size), :min(w, input_size), :] = np.array(image)[:min(h, input_size), :min(w, input_size), :3]
            
            # 이미지 저장
            img_filename = f'{base_filename}_{h1}_{w1}'
            cv2.imwrite(f'../../data/yolo_IGNITE/images/{class_name}/{img_filename}.png', pad_image)
            
            # YOLO txt 라벨 생성
            create_yolo_txt(crop_labels, img_filename, class_name)
            

print("YOLO 형식 생성 함수 준비 완료!")

In [ ]:
# 진행 전 오버랩 확인 - 실제 crop_label 함수 사용
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap
import numpy as np

def visualize_crop_overlap_with_actual_function(json_data, file_path, input_size=512, max_crops=6):
    """
    실제 crop_label 함수를 사용한 크롭 영역과 바운딩박스 오버랩 시각화
    """
    # 이미지 정보
    w = json_data['image']['width']
    h = json_data['image']['height']
    image = Image.open(os.path.join(file_path, json_data['image']['file_name']))
    
    # 크롭 계획 계산
    r = input_size / min(h, w)
    
    if r < 1:  # 이미지가 input_size보다 큰 경우
        h_count = min(h // input_size, 3)  # 최대 3x3 그리드만 표시
        w_count = min(w // input_size, 3)
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        crop_info = []
        
        for hi in range(h_count):
            for wi in range(w_count):
                if hi * w_count + wi >= max_crops:
                    break
                    
                # 크롭 위치 계산 (실제 함수와 동일한 로직)
                crop_y = hi * input_size  # h1
                crop_x = wi * input_size  # w1
                
                # 경계 조정 (실제 함수와 동일)
                if crop_y + input_size > h:
                    crop_y = h - input_size
                if crop_x + input_size > w:
                    crop_x = w - input_size
                
                # 실제 crop_label 함수 사용 🎯
                original_data = deepcopy(json_data)
                crop_labels = crop_label(crop_y, crop_x, original_data, input_size)
                
                # 원본 이미지에서 크롭 영역 추출 (PIL은 (left, top, right, bottom))
                crop_image = image.crop((crop_x, crop_y, crop_x + input_size, crop_y + input_size))
                
                crop_info.append({
                    'position': (crop_x, crop_y),
                    'image': crop_image,
                    'labels': crop_labels,
                    'crop_id': f'{hi}_{wi}'
                })
        
        # 시각화
        for idx, crop in enumerate(crop_info):
            if idx >= max_crops:
                break
                
            ax = axes[idx]
            ax.imshow(crop['image'])
            ax.set_title(f"Crop {crop['crop_id']} - {len(crop['labels'])} objects", fontsize=12, fontweight='bold')
            
            # 바운딩박스 그리기 (crop_label 함수 결과 사용)
            colors = ['red', 'blue', 'green']
            class_names = ['pd-l1 neg', 'pd-l1 pos', 'non-tumor']
            
            for label in crop['labels']:
                x, y, w, h = label['bbox']
                category = label['category_id']
                
                # 바운딩박스 그리기
                rect = patches.Rectangle((x, y), w, h, 
                                       linewidth=2, 
                                       edgecolor=colors[category-1], 
                                       facecolor='none',
                                       alpha=0.8)
                ax.add_patch(rect)
                
                # 클래스 라벨 추가
            
            ax.set_xlim(0, input_size)
            ax.set_ylim(input_size, 0)
            ax.grid(True, alpha=0.3)
        
        # 남은 subplot 숨기기
        for idx in range(len(crop_info), 6):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.suptitle(f"Actual Crop Function Analysis - {json_data['image']['file_name']}", 
                    fontsize=16, y=1.02, fontweight='bold')
        plt.show()
        
        # 상세 통계 출력
        total_crops = len([c for c in crop_info if len(c['labels']) > 0])
        total_objects = sum(len(c['labels']) for c in crop_info)
        
        print(f"📊 실제 crop_label 함수 분석 결과:")
        print(f"  - 원본 이미지 크기: {w} x {h}")
        print(f"  - 총 크롭 수: {len(crop_info)}개")
        print(f"  - 유효한 크롭 수: {total_crops}개 (객체가 있는 크롭)")
        print(f"  - 총 객체 수: {total_objects}개")
        
        # 각 크롭별 상세 정보
        print(f"\n📋 크롭별 상세 정보:")
        for i, crop in enumerate(crop_info):
            if len(crop['labels']) > 0:
                class_counts = [0, 0, 0]
                for label in crop['labels']:
                    class_counts[label['category_id']-1] += 1
                print(f"  Crop {crop['crop_id']}: {class_counts[0]}neg + {class_counts[1]}pos + {class_counts[2]}non = {len(crop['labels'])}개")
        
    else:
        # 작은 이미지의 경우 패딩 시각화 (실제 함수 사용)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        
        # 원본 이미지
        ax1.imshow(image)
        ax1.set_title("Original Image", fontsize=12, fontweight='bold')
        
        # 원본 바운딩박스 그리기
        colors = ['red', 'blue', 'green']
        for ann in json_data['annotations']:
            x, y, w, h = ann['bbox']
            category = ann['category_id']
            
            rect = patches.Rectangle((x, y), w, h, 
                                   linewidth=2, 
                                   edgecolor=colors[category-1], 
                                   facecolor='none')
            ax1.add_patch(rect)
        
        # 실제 crop_label 함수로 패딩 후 라벨 처리
        original_data = deepcopy(json_data)
        crop_labels = crop_label(0, 0, original_data, input_size)
        
        # 패딩된 이미지 생성
        pad_image = np.ones((input_size, input_size, 3), dtype=np.uint8) * 255
        pad_image[:min(h, input_size), :min(w, input_size), :] = np.array(image)[:min(h, input_size), :min(w, input_size), :3]
        
        ax2.imshow(pad_image)
        ax2.set_title(f"Padded + Cropped Labels ({len(crop_labels)} objects)", fontsize=12, fontweight='bold')
        
        # crop_label 함수 결과로 바운딩박스 그리기
        class_names = ['pd-l1 neg', 'pd-l1 pos', 'non-tumor']
        for label in crop_labels:
            x, y, w, h = label['bbox']
            category = label['category_id']
            
            rect = patches.Rectangle((x, y), w, h, 
                                   linewidth=2, 
                                   edgecolor=colors[category-1], 
                                   facecolor='none')
            ax2.add_patch(rect)
            
            # 클래스 라벨 추가
            ax2.text(x, y-5, class_names[category-1], 
                   color=colors[category-1], fontsize=8, fontweight='bold',
                   bbox=dict(boxstyle="round,pad=0.2", facecolor='white', alpha=0.7))
        
        plt.tight_layout()
        plt.show()
        
        print(f"📊 패딩 이미지 분석:")
        print(f"  - 원본 크기: {w} x {h}")
        print(f"  - 패딩 후: {input_size} x {input_size}")
        print(f"  - crop_label 함수 결과: {len(crop_labels)}개 객체")

# 샘플 이미지로 테스트
def test_overlap_visualization():
    """실제 함수 사용한 오버랩 시각화 테스트"""
    json_list = glob('../../data/IGNITE/annotations/pdl1/individual/*.json')
    file_path = '../../data/IGNITE/images/pdl1/pdl1/'
    
    if len(json_list) > 0:
        # 첫 번째 이미지로 테스트
        with open(json_list[0], 'r') as f:
            sample_data = json.load(f)
        
        print(f"🔍 실제 crop_label 함수 테스트: {sample_data['image']['file_name']}")
        print(f"📈 원본 annotation 수: {len(sample_data['annotations'])}개")
        visualize_crop_overlap_with_actual_function(sample_data, file_path, input_size=512)
    else:
        print("❌ JSON 파일을 찾을 수 없습니다!")

# 테스트 실행
test_overlap_visualization()

In [ ]:
json_list = glob('../../data/IGNITE/annotations/pdl1/individual/*.json')
file_path = '../../data/IGNITE/images/pdl1/pdl1/'

# 폴더 생성
os.makedirs('../../data/yolo_IGNITE/images/train/', exist_ok=True)
os.makedirs('../../data/yolo_IGNITE/images/val/', exist_ok=True)
os.makedirs('../../data/yolo_IGNITE/labels/train/', exist_ok=True)
os.makedirs('../../data/yolo_IGNITE/labels/val/', exist_ok=True)


# 데이터셋 생성
train_count = 0
val_count = 0

for i in tqdm(range(len(json_list)), desc="YOLO 데이터셋 생성 중"):
    # 80% train, 20% val 분할
    dataset_classification = random.randint(0, 9)
    
    with open(json_list[i], 'r') as f:
        data = json.load(f)
    
    try:
        if dataset_classification < 8:  # 80% → train
            class_name = 'train'
            custom_yolo_creation(data, file_path, input_size=512, class_name=class_name)
            train_count += 1
        else:  # 20% → val
            class_name = 'val'
            custom_yolo_creation(data, file_path, input_size=512, class_name=class_name)
            val_count += 1
    except Exception as e:
        print(f"❌ 오류 발생 ({json_list[i]}): {e}")

print(f"\n🎯 YOLO 데이터셋 생성 완료!")
print(f"📈 Train: {train_count}개 원본 이미지")
print(f"📊 Val: {val_count}개 원본 이미지")

In [ ]:
# YOLO 데이터셋 검증
def validate_yolo_dataset():
    """생성된 YOLO 데이터셋 검증"""
    
    for split in ['train', 'val']:
        img_dir = f'../../data/yolo_IGNITE/images/{split}/'
        label_dir = f'../../data/yolo_IGNITE/labels/{split}/'
        
        # 이미지와 라벨 파일 수 확인
        img_files = glob(os.path.join(img_dir, '*.png'))
        txt_files = glob(os.path.join(label_dir, '*.txt'))
        
        print(f"\n📊 {split.upper()} 데이터셋:")
        print(f"  - 이미지: {len(img_files)}개")
        print(f"  - 라벨: {len(txt_files)}개")
        
        # 라벨 통계
        total_objects = 0
        class_counts = [0, 0, 0]  # 3개 클래스
        
        for txt_file in txt_files:
            with open(txt_file, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    if line.strip():
                        class_id = int(line.split()[0])
                        class_counts[class_id] += 1
                        total_objects += 1
        
        print(f"  - 총 객체 수: {total_objects}개")
        print(f"  - Class 0 (pd-l1 negative): {class_counts[0]}개")
        print(f"  - Class 1 (pd-l1 positive): {class_counts[1]}개") 
        print(f"  - Class 2 (non-tumor): {class_counts[2]}개")

validate_yolo_dataset()

# YOLO YAML 설정 파일 생성
yaml_content = """# YOLO 데이터셋 설정
path: /home/work/IHC_biomarker/data/yolo_IGNITE
train: images/train
val: images/val
test: images/val

# 클래스 수
nc: 3

# 클래스 이름 (0부터 시작)
names:
  0: pd-l1 negative tumor cell
  1: pd-l1 positive tumor cell
  2: non-tumor cell
"""

# YAML 파일 저장
with open('IGNITE_yolo.yaml', 'w') as f:
    f.write(yaml_content)

print(f"\n✅ YOLO 설정 파일 생성: IGNITE_yolo.yaml")
print(f"🚀 이제 다음 명령으로 학습할 수 있습니다:")
print(f"   from ultralytics import RTDETR")
print(f"   model = RTDETR('rtdetr-l.pt')")
print(f"   model.train(data='IGNITE_yolo.yaml', epochs=100)")

In [ ]:
# 생성된 YOLO 데이터셋 시각화 - 이미지 + txt 파일 읽기
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random

def visualize_yolo_dataset(split='train', num_samples=6):
    """
    생성된 YOLO 데이터셋 시각화
    Args:
        split: 'train' or 'val'
        num_samples: 표시할 샘플 수
    """
    img_dir = f'../../data/yolo_IGNITE/images/{split}/'
    label_dir = f'../../data/yolo_IGNITE/labels/{split}/'
    
    # 이미지 파일 목록 가져오기
    img_files = glob(os.path.join(img_dir, '*.png'))
    
    if len(img_files) == 0:
        print(f"❌ {split} 폴더에 이미지가 없습니다!")
        return
    
    # 랜덤하게 샘플 선택
    sample_files = random.sample(img_files, min(num_samples, len(img_files)))
    
    # 서브플롯 설정
    cols = 3
    rows = (num_samples + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    if rows == 1:
        axes = [axes] if cols == 1 else axes
    else:
        axes = axes.flatten()
    
    class_names = ['pd-l1 negative', 'pd-l1 positive', 'non-tumor']
    colors = ['red', 'blue', 'green']
    
    for idx, img_path in enumerate(sample_files):
        if idx >= num_samples:
            break
            
        # 이미지 로드
        image = Image.open(img_path)
        img_filename = os.path.basename(img_path)
        txt_filename = os.path.splitext(img_filename)[0] + '.txt'
        txt_path = os.path.join(label_dir, txt_filename)
        
        # subplot 선택
        ax = axes[idx] if rows > 1 or cols > 1 else axes
        ax.imshow(image)
        
        # 해당 txt 파일 읽기
        if os.path.exists(txt_path):
            with open(txt_path, 'r') as f:
                lines = f.readlines()
            
            # YOLO 형식 파싱 및 바운딩박스 그리기
            for line in lines:
                if line.strip():
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    x_center = float(parts[1]) * 512  # 정규화 해제
                    y_center = float(parts[2]) * 512
                    width = float(parts[3]) * 512
                    height = float(parts[4]) * 512
                    
                    # 좌상단 좌표 계산
                    x = x_center - width/2
                    y = y_center - height/2
                    
                    # 바운딩박스 그리기
                    rect = patches.Rectangle((x, y), width, height,
                                           linewidth=2,
                                           edgecolor=colors[class_id],
                                           facecolor='none',
                                           alpha=0.8)
                    ax.add_patch(rect)
                    

            
            ax.set_title(f'{img_filename}\n{len(lines)} objects', fontsize=10, fontweight='bold')
        else:
            ax.set_title(f'{img_filename}\nNo labels', fontsize=10, fontweight='bold')
            print(f"⚠️ {txt_filename} 파일이 없습니다!")
        
        ax.set_xlim(0, 512)
        ax.set_ylim(512, 0)
        ax.grid(True, alpha=0.3)
    
    # 남은 subplot 숨기기
    for idx in range(len(sample_files), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f'YOLO Dataset Visualization - {split.upper()} Split', 
                fontsize=16, y=1.02, fontweight='bold')
    plt.show()

def read_and_display_txt_content(split='train', num_files=3):
    """
    txt 파일 내용을 직접 출력
    """
    label_dir = f'../../data/yolo_IGNITE/labels/{split}/'
    txt_files = glob(os.path.join(label_dir, '*.txt'))
    
    if len(txt_files) == 0:
        print(f"❌ {split} 폴더에 txt 파일이 없습니다!")
        return
    
    sample_files = random.sample(txt_files, min(num_files, len(txt_files)))
    
    print(f"📄 {split.upper()} 라벨 파일 내용 샘플:")
    print("=" * 80)
    
    for txt_path in sample_files:
        filename = os.path.basename(txt_path)
        print(f"\n📝 파일: {filename}")
        print("-" * 50)
        
        with open(txt_path, 'r') as f:
            lines = f.readlines()
        
        if len(lines) > 0:
            print("YOLO 형식: class_id x_center y_center width height")
            for i, line in enumerate(lines[:5]):  # 최대 5개만 표시
                parts = line.strip().split()
                if len(parts) == 5:
                    class_id = int(parts[0])
                    class_names = ['pd-l1 negative', 'pd-l1 positive', 'non-tumor']
                    print(f"  {i+1}: {class_names[class_id]} | {line.strip()}")
            
            if len(lines) > 5:
                print(f"  ... (총 {len(lines)}개 객체 중 5개만 표시)")
        else:
            print("  빈 파일입니다.")

def dataset_statistics():
    """
    데이터셋 전체 통계
    """
    print(f"\n📊 최종 YOLO 데이터셋 통계:")
    print("=" * 60)
    
    for split in ['train', 'val']:
        img_dir = f'../../data/yolo_IGNITE/images/{split}/'
        label_dir = f'../../data/yolo_IGNITE/labels/{split}/'
        
        img_files = glob(os.path.join(img_dir, '*.png'))
        txt_files = glob(os.path.join(label_dir, '*.txt'))
        
        # 클래스별 객체 수 계산
        total_objects = 0
        class_counts = [0, 0, 0]
        
        for txt_file in txt_files:
            with open(txt_file, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    if line.strip():
                        class_id = int(line.split()[0])
                        class_counts[class_id] += 1
                        total_objects += 1
        
        print(f"\n🎯 {split.upper()} 데이터셋:")
        print(f"  📷 이미지: {len(img_files):,}개")
        print(f"  📄 라벨: {len(txt_files):,}개")
        print(f"  🎯 총 객체: {total_objects:,}개")
        print(f"  📈 클래스 분포:")
        print(f"    - pd-l1 negative: {class_counts[0]:,}개 ({class_counts[0]/max(1,total_objects)*100:.1f}%)")
        print(f"    - pd-l1 positive: {class_counts[1]:,}개 ({class_counts[1]/max(1,total_objects)*100:.1f}%)")
        print(f"    - non-tumor: {class_counts[2]:,}개 ({class_counts[2]/max(1,total_objects)*100:.1f}%)")

# 실행
print("🎨 YOLO 데이터셋 시각화 시작")

# 1. 통계 출력
dataset_statistics()

# 2. Train 이미지 시각화
print(f"\n🖼️ Train 이미지 샘플:")
visualize_yolo_dataset('train', num_samples=6)

# 3. Val 이미지 시각화  
print(f"\n🖼️ Val 이미지 샘플:")
visualize_yolo_dataset('val', num_samples=6)

# 4. txt 파일 내용 확인
read_and_display_txt_content('train', num_files=3)
read_and_display_txt_content('val', num_files=2)

print(f"\n✅ YOLO 데이터셋 생성 및 검증 완료!")
print(f"🚀 이제 'IGNITE_yolo.yaml' 파일로 학습을 시작할 수 있습니다.")